In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.devices.experimental import TestDevicePythonSim, PlainNumpySimulator

In [2]:
from numpy import random
rng = random.default_rng(seed=42)

In [3]:
dev_old = qml.device('default.qubit', wires=2, shots=10)

In [4]:
qs = qml.tape.QuantumScript([qml.Hadamard(0), qml.Hadamard(1)], [qml.sample()])

In [5]:
dev_old.execute(qs)

array([[[1, 0],
        [1, 0],
        [0, 0],
        [0, 1],
        [0, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 1]]])

In [6]:
dev_old.sample_basis_states(4, dev_old.analytic_probability())

array([3, 0, 1, 3, 3, 3, 3, 2, 0, 1])

In [7]:
sim = PlainNumpySimulator()

In [15]:
state = sim.create_zeroes_state(2)
state = sim.apply_operation(state, qml.Hadamard(0))
state = sim.apply_operation(state, qml.CNOT((0,1)))
state 

array([[0.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.70710678+0.j]])

In [16]:
state

array([[0.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.70710678+0.j]])

In [17]:
sim.generate_samples(state, rng, 10)

array([[0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0]])